In [1]:
'''
Mount Google Drive, copy data to runtime, and unzip folders

Make sure to put a link to "EC 523 Project" in your main google drive!
'''

# from google.colab import drive
# drive.mount('/content/drive')

# ! cp /content/drive/MyDrive/'Deep Learning Proj'/train.zip /content
# ! cp /content/drive/MyDrive/'Deep Learning Proj'/test.zip /content
# ! cp /content/drive/MyDrive/'Deep Learning Proj'/val.zip /content
# ! cp /content/drive/'My Drive'/'Deep Learning Proj'/math.txt /content

# from path will differ depending on where you saved the zip file in Google Drive
# ! unzip -DD -q  ./train.zip -d  .
# ! unzip -DD -q  ./test.zip -d  .
# ! unzip -DD -q  ./val.zip -d  .



'\nMount Google Drive, copy data to runtime, and unzip folders\n\nMake sure to put a link to "EC 523 Project" in your main google drive!\n'

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
'''
Length of datasets
'''
num_train_str = !ls train | wc -l
num_test_str = !ls test | wc -l
num_val_str = !ls val | wc -l
num_train = int(num_train_str[0])
num_test = int(num_test_str[0])
num_val = int(num_val_str[0])

print(f'Number of train images: {num_train}\nNumber of test images: {num_test}\nNumber of validation images: {num_val}\nTotal images: {num_train+num_test+num_val}')

Number of train images: 158480
Number of test images: 30637
Number of validation images: 6765
Total images: 195882


In [4]:
train_root = "./train/"
test_root = "./test/"
val_root = "./val/"
label_file = "./math.txt"
freq_vocabs = "./Corpus.txt"

Creating dataset class for images and labels

In [5]:
import os
import cv2
import torch.utils.data
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np


class LatexDataset(torch.utils.data.Dataset):
  def __init__(self, transform=None, dataroot=val_root, max_seq_length = 700): # can change dataroot to be either train_root, test_root, val_root
        '''Initialize the dataset.'''
        self.transform = transform
        self.dataroot = dataroot
        self.labels_txt = label_file
        self.max_seq_length = max_seq_length
        self._parse()

  def _parse(self):
        '''
        Parse the math.txt file.
        Populates the following private variables:
        - self.im_paths: A list of strings storing the associated image paths
        - self.labels: A list of strings, where each string is the latex code for an image
        '''
        def getImPath(idx):
            # Find image in either train, test, or validation folder
            imname = str(idx - 1).zfill(7) + '.png'
            if os.path.exists(f'{self.dataroot}{imname}'):
              impath = f'{self.dataroot}{imname}'
            else:
              return None

            try:
                Image.open(impath).verify()
            except Exception as e:
                # Some images can't be opened
                # print(f"Image at path {impath} is corrupted. Error: {e}")
                return None

            return impath

        self.im_paths = []
        self.labels = []

        with open(self.labels_txt) as f:
            for idx, line in enumerate(f):
                impath = getImPath(idx+1)

                if impath is not None:
                    labels = line.strip('\n')
                    if len(labels) < self.max_seq_length-1:   # Loading images with certain latex length
                      self.im_paths.append(impath)            # Image name
                      self.labels.append(labels)    # String of latex code


  def __len__(self):
        '''Return length of the dataset.'''
        assert len(self.labels) == len(self.im_paths)
        return len(self.labels)

  def __getitem__(self, index):
        '''
        Return the (image, attributes) tuple.
        This function gets called when you index the dataset.
        '''
        def img_load(index):
            imraw = Image.open(self.im_paths[index])
            imgray = imraw.convert('L')                         # Convert image to greyscale
            imthresh = imgray.point(lambda p: p > 240 and 255)  # Threshold image to remove background (white)
            if self.transform is not None:
              im = self.transform(imthresh)
            else:
              im = imthresh
            return im

        target = self.labels[index]
        return img_load(index), target

Generating most frequent characters used in math.txt file and outputing it to output.txt

In [6]:
from collections import Counter

def create_dictionary(file_path, n, output_file_path):
    word_count = Counter()

    with open(file_path, 'r') as file:
        # Read the file and count word occurrences
        for line in file:
            words = line.split()
            word_count.update(words)

    # Get the n most common words as a dictionary
    most_common_words = dict(word_count.most_common(n))

    # Save the dictionary to a text file
    ID = 1
    with open(output_file_path, 'w') as output_file:
        for word, count in most_common_words.items():
          if (word != ',') and (word != '\\,'):
            output_file.write(f"{word},{ID} \n")
            ID += 1
        output_file.write(f"{'PAD'},{ID} \n")
        ID += 1
        output_file.write(f"{'UNK'},{ID} \n")

# Example usage
n_most_common = 200  # Change this value to get different numbers of most common words
!touch ./output.txt
output_file_path = "./output.txt" # Replace 'output.txt' with the desired output file path

create_dictionary(label_file, n_most_common, output_file_path)
print(f"Data saved to {output_file_path}")

Data saved to ./output.txt


Creating dictionary for vocab and tokens

In [7]:
'''
Dictionary block: converts a LaTeX string to a dictionary of latex tokens, where
each unique token has its own entry and integer value assigned to it

'''
class LatexDict():
    def __init__(self, max_seq_length=256):
        self.labels_txt = label_file
        self.max_seq_length = max_seq_length
        self.latex_dict = {'<UKN>':0, '<PAD>':1} # Initialize with token for unknown and for padding
        self.latex_dict_inverse = {0:'<UKN>', 1:'<PAD>'} # Initialize inverse dict for quicker reverse lookups
        self.create_dict()

    def create_dict(self):
        # Go through entire label file and populate dictionary
        with open(self.labels_txt) as f:
            for line in f:
                tokens = line.split()
                for token in tokens:
                    if token not in self.latex_dict:
                        # Assign a new ID for the unseen token
                        new_id = len(self.latex_dict)
                        self.latex_dict[token] = new_id
                        self.latex_dict_inverse[new_id] = token

    def map_tokens(self, tex_str_list, batch_size):
        ids_tensor = torch.full((batch_size, self.max_seq_length), self.latex_dict['<PAD>'], dtype=torch.float32)

        for row, tex_str in enumerate(tex_str_list):
            tex_str = r'{ ' + tex_str + ' }'
            tokens = tex_str.split()
            for col, token in enumerate(tokens):
                ids_tensor[row, col] = self.latex_dict[token]

        return ids_tensor

    def tokens_to_tex(self, token_vec):
        tex_str = ' '
        for token_id in token_vec.tolist():
            if token_id in self.latex_dict_inverse:
                if self.latex_dict_inverse[token_id] != '<PAD>' and self.latex_dict_inverse[token_id] != '<UKN>':
                    tex_str += self.latex_dict_inverse[token_id] + ' '

        return tex_str

    def __dict__(self):
        return self.latex_dict

    def __len__(self):
        return len(self.latex_dict)

latex_dict = LatexDict()

LatexDictModified() is debugged version of LatexDict(). The problems with loading labels and dealing with escape characters (e.g \n, \t) have been resolved. It also puts UNK for each unknown character

In [8]:
'''
Dictionary block: converts a LaTeX string to a dictionary of latex tokens, where
each unique token has its own entry and integer value assigned to it

'''
class LatexDictModified():
    def __init__(self, max_seq_length=256):
        self.labels_txt = label_file
        self.max_seq_length = max_seq_length
        self.latex_dict = {} # Initialize with token for unknown and for padding
        self.latex_dict_inverse = {} # Initialize inverse dict for quicker reverse lookups
        self.create_dict()

    def create_dict(self):
        # Go through entire label file and populate dictionary

      Freq_labeled_file = open("./output.txt", "r")

      for line in Freq_labeled_file: #For each line in data file
          token = line.split(',')[1].split(" ")[0]
          latexSymbol = line.split(",")[0]
          self.latex_dict[latexSymbol] = int(token)
          self.latex_dict_inverse[token] = latexSymbol
      Freq_labeled_file.close()

    def map_tokens(self, tex_str_list, batch_size):
        ids_tensor = torch.full((batch_size, self.max_seq_length), self.latex_dict['PAD'], dtype=torch.float32)

        for row, tex_str in enumerate(tex_str_list):
            tex_str = r'{ ' + tex_str + ' }'
 
            tokens = tex_str.split()

            for col, token in enumerate(tokens):
              if token in self.latex_dict:
                ids_tensor[row, col] = self.latex_dict[token]
              else:
               ids_tensor[row, col] = self.latex_dict['UNK']

        return ids_tensor

    def tokens_to_tex(self, token_vec):
        tex_str = ' '
        for token_id in token_vec.tolist():
            if token_id in self.latex_dict_inverse:
                if self.latex_dict_inverse[token_id] != 'PAD' and self.latex_dict_inverse[token_id] != 'UNK':
                    tex_str += self.latex_dict_inverse[token_id] + ' '

        return tex_str

    def __dict__(self):
        return self.latex_dict

    def __len__(self):
        return len(self.latex_dict)

latex_dict = LatexDictModified()

CNN Block with 512 channel output

In [76]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
import math

class CNN_Block(nn.Module):
    def __init__(self):
        super(CNN_Block, self).__init__()
        
        self.leaky = nn.LeakyReLU(negative_slope=0.1)
        # self.conv1 = nn.Conv2d(3, 64, 3)
        self.conv1 = nn.Conv2d(1, 64, 3, padding=1, bias=False)    # Images are originally one channel, added padding as well
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(128)
        # self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1, bias=False)
        # self.pool3 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(256, 256, 3, padding=1, bias=False)
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(512)
        # self.pool4 = nn.MaxPool2d(2, 2)

        #self.fc1 = nn.Linear(256 * 2 * 8, 1024)  # Adjusted input size based on the output size of the convolutional layers
        #self.fc2 = nn.Linear(1024, 512)


    def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = self.pool(F.relu(self.conv3(x)))
#         x = (F.relu(self.conv4(x)))
        # x = F.relu(self.conv5(x))
    
        ''' 
        Replacing Leaky ReLU instead of ReLU
        '''
        
        x = self.pool(self.leaky(self.conv1(x)))
        x = self.pool(self.bn1(self.leaky(self.conv2(x))))
        x = self.pool(self.leaky(self.conv3(x)))
        x = self.pool(self.leaky(self.conv4(x)))
        x = self.bn2(self.leaky(self.conv5(x)))
    
        #print('x: shape:', x.shape)
        #x = x.view(64,-1)   # Flatten so this can be used in linear layers

        #x = F.relu(self.fc1(x))
        #x = self.fc2(x)

        return x


Defining decoding class: \\
  hidden_size: dimensionality of the hidden layer of the LSTM \\
  batch_size: Size of the batch \\
  seq_length: length of the input sequence \\
  vocab_size: size of our dictionary \\
  emd_size: Number of rows in embedding vector to represent \\
  enc_out: Number of output channels in encoder \\
  

In [84]:
class decoder(nn.Module):
  def __init__(self, hidden_size, batch_size, seq_length, vocab_size, embedding_size, enc_out = 512):
    super(decoder, self).__init__()

    self.hidden_size = hidden_size
    self.batch_size = batch_size
    self.seq_length = seq_length
    self.vocab_size = vocab_size
    self.embedding_size = embedding_size
    self.enc_out = enc_out


    self.cnn_encoder = CNN_Block()
    self.lstm = nn.LSTMCell(hidden_size + embedding_size, hidden_size)
    self.emb = nn.Embedding(vocab_size,embedding_size )

    self.wh_in = nn.Linear(enc_out, hidden_size, bias = False)
    self.wc_in = nn.Linear(enc_out, hidden_size, bias = False)
    self.wo_in = nn.Linear(enc_out, hidden_size, bias = False)

    self.dropout = nn.Dropout(p=0.2)
    self.soft = nn.Softmax(dim = 2)
    self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

 # Attention mechanism
    self.beta = nn.Parameter(torch.Tensor(enc_out))
    nn.init.uniform_(self.beta, -1e-2, 1e-2)
    self.W_1 = nn.Linear(enc_out, enc_out, bias=False)
    self.W_2 = nn.Linear(hidden_size, enc_out, bias=False)

    self.W_3 = nn.Linear(hidden_size + enc_out, hidden_size, bias=False)
    self.W_out = nn.Linear(hidden_size, vocab_size, bias=False)


  def forward(self, im, label):
    enc_im = self.encode(im.to(self.device))
    dec_states, o_t = self.init_decoder(enc_im.to(self.device))
    # Suppose we are taking labels with format [Batch, Max Len, Vocab_size]
    max_len = label.shape[1]
    outputs = []
    flag = False
    for i in range(max_len):
      lll = label[:, i, :]
      target_label = torch.argmax(lll.to(self.device), dim = 1, keepdim = True)
      if flag:
        target_label = torch.argmax(outputs[-1].to(self.device), dim = 1, keepdim = True) # Finding the result of the previous output and feed it to the next input

      dec_states, output = self.step_decode(dec_states, o_t.to(self.device), enc_im.to(self.device), target_label.to(self.device))
      outputs.append(output.to(self.device))
      flag = True
    outputs = torch.stack(outputs, dim=1) # Changing the dimensions to [Batch, Max Len, Vocab_size]
    outputs = self.soft(outputs)

    return outputs




  def encode(self, im):
    cnn_encode = self.cnn_encoder(im.to(self.device))
    
    pe = self.PositionalEmbedding2D(self.enc_out, cnn_encode.shape[2], cnn_encode.shape[3]).repeat(cnn_encode.shape[0], 1, 1, 1).to(self.device)

    cnn_encode = cnn_encode.permute(0, 2, 3, 1)   # [Batch, Width, Height, Channel]
    batch, height, width, channel = cnn_encode.shape


    cnn_encode = cnn_encode.contiguous().view(batch, height * width, -1)  # [Batch, Width*Height, Channel]

    return cnn_encode


  def PositionalEmbedding2D(self, D_model,height,width):
    if D_model % 4 != 0:
        raise ValueError("Cannot use sin/cos positional encoding with "
                         "odd dimension (got dim={:d})".format(D_model))
    pe = torch.zeros(D_model,height,width)
    d_model = int(D_model / 2)
    div_term = torch.exp(torch.arange(0., d_model, 2) * -(math.log(10000.0) / d_model))
    pos_w = torch.arange(0., width).unsqueeze(1)
    pos_h = torch.arange(0., height).unsqueeze(1)
    pe[0:d_model:2, :, :] = torch.sin(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
    pe[1:d_model:2, :, :] = torch.cos(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
    pe[d_model::2, :, :] = torch.sin(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)
    pe[d_model + 1::2, :, :] = torch.cos(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)

    return pe.permute(0, 1, 2)


  def init_decoder(self, enc_im):
    '''
    Here we are trying to estimate if we had a LSTM module for encoder, what would be the result of H, C, O
    '''
    enc_im_mean = enc_im.mean(dim=1)
    h = torch.tanh(self.wh_in(enc_im_mean))
    c = torch.tanh(self.wc_in(enc_im_mean))
    o_init = torch.tanh(self.wo_in(enc_im_mean))
    return (h, c), o_init



  def step_decode(self, dec_states, o_t, enc_im, target_label ):
    y_emb = self.emb(target_label).squeeze(1) 
    inp = torch.cat([y_emb, o_t], dim = 1)
    h, c = self.lstm(inp, dec_states)
    h = self.dropout(h)
    c = self.dropout(c)

    contex = self.attentionMechanism(enc_im, h)

    o_t = self.W_3(torch.cat([h, contex], dim=1)).tanh()
    o_t = self.dropout(o_t)
    output = self.W_out(o_t)

    return (h, c), output


  def attentionMechanism(self, enc_im, h):
    alpha = torch.tanh(self.W_1(enc_im) + self.W_2(h).unsqueeze(1))
    alpha = torch.sum(self.beta * alpha, dim=-1)  # [B, L]
    alpha = F.softmax(alpha, dim=-1)  # [B, L]

    # cal context: [B, C]
    contex = torch.bmm(alpha.unsqueeze(1), enc_im)
    contex = contex.squeeze(1)
    return contex



Initializing the database and code output

In [18]:
'''
Initialize dataset and image preprocessing

NOTE:
    Some of the images in the dataset are corrupted. To deal with this,
    there is a check for each image to ensure that it can be loaded.
'''
import torchvision.transforms as transforms

reduced_imsize = (32, 128)  # Images are reduced to this size

# Define the transform pipeline - add normalization?
transform = transforms.Compose([
    transforms.Resize(reduced_imsize),
    transforms.ToTensor(),
])


seq_length = 150        # Maximum number of sequence length i

train_dataset = LatexDataset(transform=transform, dataroot=train_root, max_seq_length=seq_length)
test_dataset = LatexDataset(transform=transform, dataroot=test_root)
val_dataset = LatexDataset(transform=transform, dataroot=val_root)

# Device settings
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

Device: cuda:0


In [88]:
'''
Initialize hyperperameters, trainloader, and dictionary of LaTeX token mappings
'''

# Hyperparameters
batch_size = 16
learning_rate = 0.001
weight_decay = 0  # (L2 penalty)

latex_dict = LatexDictModified(max_seq_length=seq_length)

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)


In [89]:
'''
Initialize Model

Initialize Loss Functions:
1. Normal Cross-Entropy Loss between prediction and label
2. LaTeX compile test:
    - Custom function, returns True if code can compile into LaTeX, False if not

Initialize Optimizer:
1. Adam Optimizer
'''

    # Hyperparameters


hidden_size = 200
batch_size = 16
vocab_size = latex_dict.__len__()
embedding_size = 80
model = decoder(hidden_size, batch_size, seq_length, vocab_size, embedding_size).to(device)

# model = Model(embedding_size=embedding_size, hidden_size=hidden_size, batch_size=batch_size, sequence_length=sequence_length, vocab_size=vocab_size, o_layer_size = o_layer_size).to(device)
criterion = nn.CrossEntropyLoss()   ## May need to change this
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

Latex Checker

Install Dependencies

In [ ]:
!pip install folium==0.2.1
!pip install pdflatex
!sudo apt-get install texlive-latex-recommended 
!sudo apt install texlive-latex-extra
!sudo apt install dvipng

Function: 

In [ ]:
import subprocess
import pdflatex as ptex
def check_latex_syntax(tex_code):
    try:
        # Create a .tex file with the provided LaTeX code
        with open('temp.tex', 'w') as file:
            file.write(tex_code)

        # Run pdflatex to compile the temporary .tex file
        subprocess.run(['pdflatex', '-interaction=batchmode', 'temp.tex'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        # Check the return code to determine if compilation was successful
        return_code = subprocess.call(['pdflatex', '-interaction=batchmode', 'temp.tex'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        if return_code == 0:
            return True  # Compilation successful, syntax is valid
        else:
            return False  # Compilation failed, syntax is invalid

    except subprocess.CalledProcessError:
        return False

    finally:
        # Clean up temporary files
        subprocess.run(['rm', 'temp.tex', 'temp.log', 'temp.aux'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)


#Counting the failed in compiling
def count_latex_errors(tex_code):
  Error_count = 0
  for i in range(len(tex_code)):
    temp_text = tex_code[i]
    result = check_latex_syntax(temp_text)
    if result:
      pass
    else:
      Error_count += 1
  return Error_count
def making_input_syntax_check(tex_str_list):
  Latex_code = []
  Base_start = r'''
\documentclass{article}
\begin{document}
'''
  Base_end = r'''
\end{document}
'''
  for i in range(len(tex_str_list)):
    temp_tex_str = Base_start + '$' + tex_str_list[i] + '$' + Base_end
    Latex_code.append(temp_tex_str)
  return Latex_code

Usage Example

In [ ]:
latex_code = r'''\beta'''
latex = r'''\frac{a}{b}'''

tex_code =[]
tex_code.append(latex_code)
tex_code.append(latex)
tex_code_correct = making_input_syntax_check(tex_code)
# Check the LaTeX syntax
print(count_latex_errors(tex_code_correct))

Training Loop

In [87]:
from tqdm import tqdm

'''
Training Loop
'''




num_epoch = 100
model.train()
counter = 0
for epoch in range(num_epoch):
    print('epoch:', epoch)
    pbar = tqdm(trainloader)
    for images, y in pbar:

        images = images.to(device)              # Send to gpu
  
        y_vec = latex_dict.map_tokens(list(y), batch_size=batch_size)
        y_vec = torch.tensor(y_vec, dtype=torch.long)
        y_vec = torch.LongTensor(y_vec)
  
        one_hot = torch.zeros(y_vec.shape[0], y_vec.shape[1], vocab_size)
        for i in range(y_vec.shape[0]):     # Making one-hot representation of the labels [Batch, Max Len, Vocab-Size]
            for j in range(y_vec.shape[1]):
                one_hot[i,j,y_vec[i,j]-1] = 1
        one_hot = one_hot.to(device)                # Send to gpu
      
        predictions = model.forward(images,one_hot)        # Get predictions


        one_hot = torch.tensor(one_hot, dtype=torch.float)


        counter += 1
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        loss1 = criterion(predictions, one_hot)   # Calculate first loss function
        print(loss1)
        if (counter % 50 == 0):
            
            counter = 1
            print("===================Checking gradient===============")
            for name, param in model.named_parameters():
                print(name, param.grad.norm())
            print("===================    Finished   =================")
#             with open('./loss_report.txt', 'a') as output_file:
#                     output_file.write(f"{epoch},{loss1} \n")
        # loss2 = can_compile(precitions)         # Check if output can compile

        optimizer.zero_grad()

        loss1.backward()
        optimizer.step()
#     torch.save(model.state_dict(), f'checkpoint_{epoch}.pt')

epoch: 0


  0%|          | 0/1529 [00:00<?, ?it/s]/scratch/2885366.1.ece/ipykernel_2656175/2454034753.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_vec = torch.tensor(y_vec, dtype=torch.long)
/scratch/2885366.1.ece/ipykernel_2656175/2454034753.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  one_hot = torch.tensor(one_hot, dtype=torch.float)
  0%|          | 1/1529 [00:00<18:05,  1.41it/s]

tensor(3.7580, device='cuda:0', grad_fn=<DivBackward1>)



  0%|          | 2/1529 [00:01<13:33,  1.88it/s]

tensor(3.7580, device='cuda:0', grad_fn=<DivBackward1>)


...

tensor(3.6673, device='cuda:0', grad_fn=<DivBackward1>)



 28%|██▊       | 423/1529 [02:18<05:33,  3.31it/s]

tensor(3.6816, device='cuda:0', grad_fn=<DivBackward1>)


 28%|██▊       | 423/1529 [02:19<06:03,  3.04it/s]


KeyboardInterrupt: 